In [146]:
import requests
import os

Day = 14

# get file from website using private session key stored in enviromental variables
r = requests.get(
            f'https://adventofcode.com/2023/day/'+str(Day)+'/input',
            cookies={'session': os.getenv('AdventSessionKey')}
)

# read r.text
data = r.text.strip().split('\n')


In [157]:
# Functions

def count_Os(input_string):
    # we are rotating the input becuase it is easier to work on rows
    transposed = list(map(''.join, zip(*input_string)))

    # Partition by hashes and count 'O's
    counts = []
    positions = []
    
    for line in transposed:
        # split into groups and count O's each each group
        groups = line.split('#') 
        counts.append([group.count('O') for group in groups])

        # Calculate starting positions of the hashes
        # This helps us rebuild the board later
        pos = [i+1 for i, c in enumerate(line) if c == '#']
        pos = [0] + pos  # Always start with a zero
        positions.append(pos)

    return counts, positions

def build_board(counts, positions, column_length):
    board = []
    for count_list, position_list in zip(counts, positions):
        column = ['.' for _ in range(column_length)]
        for count, position in zip(count_list, position_list):
            if position != 0:
                column[position - 1] = '#'
            for i in range(count):
                column[position + i] = 'O'
        board.append(''.join(column))
    # Transpose the board to put columns side by side
    transposed = list(map(list, zip(*board)))
    # Join each row into a string
    return [''.join(row) for row in transposed]

def calculate_sum(board, column_length):
    total = 0
    for i, row in enumerate(board):
        count_O = row.count('O')
        total += count_O * (column_length - i)
    return total

def rotate_board(board):
    # Rotate the board
    rotated = list(map(''.join, zip(*board)))
    # Reflect each row
    return [row[::-1] for row in rotated]

def do_one_cycle(board, column_length):

    for i in range(4): # N, S, E, W

        # get info to tilt the board
        counts, positions = count_Os(board)
        # tilt the board
        board = build_board(counts, positions, column_length)
        # Rotate the board
        board = rotate_board(board)
    
    return board

def find_cycle(board, column_length):
    # Convert board to string
    board_str = '\n'.join(board)
    
    # Store each unique state and the step at which it was first seen
    seen = {board_str: 0}
    
    for step in range(1, 1000000000):
        # Do one cycle
        board = do_one_cycle(board, column_length)
        
        # Convert board to string
        board_str = '\n'.join(board)
                
        # If the current state has been seen before, we've found a cycle
        if board_str in seen:
            cycle_length = step - seen[board_str] # seen board started the cycle
            remaining_steps = (1000000000 - step) % cycle_length
            final_board = [b for b, v in seen.items() if v == seen[board_str] + remaining_steps][0]
            
            return final_board.split('\n')  # Convert final board back to list
        
        # Not seen yet, add to the seen list
        seen[board_str] = step
                
    return board



In [144]:
# Test

input_string = [
'O....#....',
'O.OO#....#',
'.....##...',
'OO.#O....O',
'.O.....O#.',
'O.#..O.#.#',
'..O..#O..O',
'.......O..',
'#....###..',
'#OO..#....'
]

# get info to tilt the board
counts, positions = count_Os(input_string)

column_length = len(input_string[0])  # Length of a column

# tilt the board
board = build_board(counts, positions, column_length)
# print(board)

total = calculate_sum(board, column_length)
print("Total:", total)

# 136

Total: 136


In [148]:
# Part 1

# get info to tilt the board
counts, positions = count_Os(data)

# Length of a column
column_length = len(data[0])

# tilt the board
board = build_board(counts, positions, column_length)
# print(board)

total = calculate_sum(board, column_length)
print("Total:", total)

# 109755


Total: 109755


In [158]:
# Test
input_string = [
'O....#....',
'O.OO#....#',
'.....##...',
'OO.#O....O',
'.O.....O#.',
'O.#..O.#.#',
'..O..#O..O',
'.......O..',
'#....###..',
'#OO..#....'
]

column_length = len(input_string[0])  # Length of a column

final_board = find_cycle(input_string,column_length)
total = calculate_sum(final_board,column_length)
print("Total:", total)

# 64

Total: 64


In [159]:
# Part 2

column_length = len(data[0])  # Length of a column

final_board = find_cycle(data,column_length)
total = calculate_sum(final_board,column_length)
print("Total:", total)

Total: 90928
